In [41]:
%matplotlib inline

import random
import math
import numpy as np
import matplotlib.pyplot as plt

In [42]:
from graphviz import Digraph

def trace(root):
    # builds a set of all nodes and edges in a graph
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root):
    dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR = left to right
    
    nodes, edges = trace(root)
    for n in nodes:
        uid = str(id(n))
        # for any value in the graph, create a rectangular ('record') node for it
        dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
        if n._op:
            # if this value is a result of some operation, create an op node for it
            dot.node(name = uid + n._op, label = n._op)
            # and connect this node to it
            dot.edge(uid + n._op, uid)

    for n1, n2 in edges:
        # connect n1 to the op node of n2
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)

    return dot

In [43]:
class Value:
    
    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.label = label

    def __repr__(self):
        return f"Value(data={self.data})"

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward

        return out
    
    def __radd__(self, other): # other + self
        return self + other
    
    def __neg__(self): # -self
        return self * -1
    
    def __sub__(self, other): # self - other
        return self + (-other)

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward

        return out
    
    def __rmul__(self, other): # other * self
        # a fallback for operations like 2 * a (since python calls 2.__mul__(a) which is not defined)
        return self * other
    
    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        out = Value(self.data ** other, (self,), f'**{other}')

        def _backward():
            self.grad += (other * self.data ** (other - 1)) * out.grad
        out._backward = _backward
        
        return out
    
    def __truediv__(self, other):
        return self * other**-1
    
    def tanh(self):
        n = self.data
        t = (math.exp(2*n) - 1) / (math.exp(2*n) + 1)
        out = Value(t, (self, ), 'tanh')

        def _backward():
            self.grad += (1 - t ** 2) * out.grad
        out._backward = _backward

        return out

    
    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self, ), 'exp')

        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward
        
        return out

    def backward(self):
        topo = []
        visited = set()

        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

In [91]:
class Neuron:

    def __init__(self, nin):
        self.w = [Value(random.uniform(-1, 1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1, 1))

    def __call__(self, x):
        # w * x  + b
        act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
        out = act.tanh()
        return out
    
    def parameters(self):
        return self.w + [self.b]
    
class Layer:

    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]

    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs
    
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]
    
class MLP:

    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

In [92]:
x = [2.0, 3.0, -1.0]
n = MLP(3, [4, 4, 1])
n(x)

Value(data=0.07265793761207102)

In [85]:
len(n.parameters())

41

In [86]:
# draw_dot(n(x))

In [88]:
xs = [
  [2.0, 3.0, -1.0],
  [3.0, -1.0, 0.5],
  [0.5, 1.0, 1.0],
  [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0] # desired targets
ypred = [n(x) for x in xs]
ypred

[Value(data=-0.9217232480328036),
 Value(data=-0.891865787802085),
 Value(data=-0.8277865536541911),
 Value(data=-0.902026339978513)]

In [66]:
loss = sum((yhat - ytrue) **2 for ytrue, yhat in zip(ys, ypred))
loss

Value(data=4.358514947131309)

In [67]:
loss.backward()

In [70]:
n.layers[0].neurons[0].w[0].grad

-0.08452109222156788

In [71]:
n.layers[0].neurons[0].w[0].data

0.09554374378014008

one manual step

In [72]:
for p in n.parameters():
    p.data -= 0.01 * p.grad

In [73]:
n.layers[0].neurons[0].w[0].data

0.09638895470235576

In [74]:
ypred = [n(x) for x in xs]
loss = sum((yhat - ytrue) **2 for ytrue, yhat in zip(ys, ypred))
loss

Value(data=4.014122099746351)

## Automate training

In [113]:
x = [2.0, 3.0, -1.0]
n = MLP(3, [4, 4, 1])
n(x)

Value(data=-0.4743421692468671)

In [114]:
for k in range(20):

    # forward pass
    ypred = [n(x) for x in xs]
    loss = sum((yhat - ytrue) **2 for ytrue, yhat in zip(ys, ypred))

    for p in n.parameters():
        p.grad = 0.0 # zero grad

    # backward pass
    loss.backward()

    # update
    for p in n.parameters():
        p.data -= 0.1 * p.grad
    
    print(k, loss.data)

0 5.753535166123903
1 2.752212290705899
2 2.1868072280673667
3 1.6974898571391248
4 1.2243321875354403
5 0.8978074545012453
6 0.49925140392839246
7 0.1596707625978528
8 0.11254428044923859
9 0.08968868078903709
10 0.07421549505804102
11 0.0629505003310351
12 0.05441585509148121
13 0.047757291043977114
14 0.04243859003457438
15 0.038106240844761115
16 0.034518480222552335
17 0.031504938344931246
18 0.028942463455391827
19 0.026740054905189103


In [115]:
# desired targets
# [1.0, -1.0, -1.0, 1.0]

ypred

[Value(data=0.9157169042587502),
 Value(data=-0.9326719755788915),
 Value(data=-0.9149391930660962),
 Value(data=0.9112981909498787)]